# introduct python3.8 environment 

In [10]:
#conda create --name py38_env --override-channels -c https://repo.anaconda.com/pkgs/snowflake python=3.8 numpy pandas

# install packages

In [11]:
pip install snowflake-snowpark-python
pip install "snowflake-connector-python[pandas]"
pip install pyspark




SyntaxError: invalid syntax (<ipython-input-11-b794ec6d90d6>, line 1)

In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
import pyspark
from pyspark.sql import SparkSession
import pyarrow
import snowflake.connector
import pandas as pd

# create a session for the snowpark (connection)

In [2]:
connection_parameters = {
...    "account": "xxxxx",
...    "user": "xxxx",
...    "password": "xxxx",
...    "role": "SYSADMIN",  # optional
...    "warehouse": "COMPUTE_WH",  # optional
...    "database": "IEDM1DB",  # optional
...    "schema": "PROJECT",  # optional
...  }  


# create a session to connect to Snowflake with connection parameters
# recall this session when we need to read, transform, store data in Snowflake.

In [4]:
session = Session.builder.configs(connection_parameters).create() 

In [5]:
orders = session.table("orders")
departments = session.table("departments")
products = session.table("products")
aisles = session.table("aisles")
op_prior = session.table("order_products_prior")
op_train = session.table("order_products_train")

In [5]:
type(orders)

snowflake.snowpark.table.Table

In [11]:
order_df = orders.limit(100000).toPandas()
department_df = departments.toPandas()
product_df = products.toPandas()
aisle_df = aisles.toPandas()
opTrain_df = op_train.limit(100000).toPandas()
opPrior_df = op_prior.limit(100000).toPandas()

In [8]:
op = orders.join(op_prior, ["order_id"])

In [9]:
op_df = op.toPandas()

In [10]:
type(op_df)

pandas.core.frame.DataFrame

In [13]:
print(op_df)

          ORDER_ID  USER_ID EVAL_SET  ORDER_NUMBER  ORDER_DOW  \
0          1555909   150314    prior             9          2   
1          1555910   109164    prior            33          0   
2          1555910   109164    prior            33          0   
3          1555910   109164    prior            33          0   
4          1555911    95605    prior            14          0   
...            ...      ...      ...           ...        ...   
32434484   1555909   150314    prior             9          2   
32434485   1555909   150314    prior             9          2   
32434486   1555909   150314    prior             9          2   
32434487   1555909   150314    prior             9          2   
32434488   1555909   150314    prior             9          2   

          ORDER_HOUR_OF_DAY  DAYS_SINCE_PRIOR_ORDER  PRODUCT_ID  \
0                        13                     3.0       12658   
1                        16                     7.0       20448   
2                 

In [14]:
# user predictor 
user = op_df.groupby("USER_ID")["ORDER_NUMBER"].count().to_frame("user_total_orders")

In [15]:
# product predictor
products = op_df.groupby('PRODUCT_ID')['ORDER_NUMBER'].count().to_frame('product_total_purchase')

In [16]:
# user product predictor
user_product = op_df.groupby(['USER_ID','PRODUCT_ID'])['ORDER_ID'].count().to_frame('user_total_on_product')

In [17]:
user_product = user_product.reset_index()

In [18]:
merge_data1 = user_product.merge(user, on = 'USER_ID', how = 'left')

In [19]:
merge_data2 = merge_data1.merge(products, on = 'PRODUCT_ID', how = 'left')

# Train data

In [20]:
order_selected = order[['USER_ID','EVAL_SET','ORDER_ID']]

In [21]:
merge_data3 = merge_data2.merge(order_selected[:100000], on = 'USER_ID', how = 'left')

In [22]:
data = merge_data3.dropna()

In [23]:
set(data['EVAL_SET'])

{'prior', 'test', 'train'}

In [24]:
train = data[data.EVAL_SET == 'train']

In [25]:
len(train)

248905

In [26]:
opTrain_selected = opTrain[['PRODUCT_ID','ORDER_ID','REORDERED']]

In [27]:
opTrain_dropdu = opTrain_selected.drop_duplicates()

In [28]:
opTrain_dropna = opTrain_dropdu.dropna()

In [29]:
train_merge1 = train.merge(opTrain_dropna[:100000], on = ['PRODUCT_ID'], how='left')

In [30]:
print(train_merge1)

          USER_ID  PRODUCT_ID  user_total_on_product  user_total_orders  \
0          155035        1263                      1                 25   
1          155035        1263                      1                 25   
2          155035        1263                      1                 25   
3          155035        1263                      1                 25   
4          155035        1263                      1                 25   
...           ...         ...                    ...                ...   
16670168   161135       49235                      1                 93   
16670169   161135       49235                      1                 93   
16670170   161135       49235                      1                 93   
16670171   161135       49235                      1                 93   
16670172   161135       49235                      1                 93   

          product_total_purchase EVAL_SET  ORDER_ID_x  ORDER_ID_y  REORDERED  
0                   

# test data

In [ ]:
test = data[data.EVAL_SET == 'test']

In [ ]:
test_setindex = test.set_index(['USER_ID', 'PRODUCT_ID'])

In [ ]:
test_selected = test.drop(['EVAL_SET','ORDER_ID'], axis=1)

# MODEL

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
x_train, y_train = train_merge1.drop('REORDERED', axis=1), train_merge1.REORDERED

In [ ]:
log = LogisticRegression(random_state=42)

In [ ]:
#model = log.fit(x_train, y_train)

In [ ]:
#session.close() 